In [1]:
pip install tensorflow==1.0.0

     |████████████████████████████████| 44.5MB 98kB/s 
  Found existing installation: tensorflow 2.4.1
    Uninstalling tensorflow-2.4.1:
      Successfully uninstalled tensorflow-2.4.1


In [2]:
import tensorflow as tf
x_data = [[1,2,1,1], [2,1,3,2], [3,1,3,4], [4,1,5,5], [1,7,5,5], [1,2,5,6], [1,6,6,6], [1,7,7,7]]
y_data = [ [0,0,1], [0,0,1], [0,0,1], [0,1,0], [0,1,0], [0,1,0], [1,0,0], [1,0,0]]
nb_classes = 3
X = tf.placeholder("float", [None, 4])
Y = tf.placeholder("float", [None, 3])
W = tf.Variable(tf.random_normal([4, nb_classes]), name ='weight')
b = tf.Variable(tf.random_normal([nb_classes], name = 'bias'))
# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
hypothesis = tf.nn.softmax(tf.matmul(X,W) + b)

# Cross entropy cost/loss
cost = tf.reduce_mean(-tf.reduce_sum(Y + tf.log(hypothesis), axis = 1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.1).minimize(cost)

# Launch graph
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for step in range(2001) :
    sess.run(optimizer, feed_dict = {X: x_data, Y: y_data})
    if step % 200 == 0:
      print(step, sess.run(cost, feed_dict={X: x_data, Y: y_data}))
  all = sess.run(hypothesis, feed_dict = {X: [[1,11,7,9], [1,3,4,3], [1,1,0,1]]})
  print(all, sess.run(tf.arg_max(all, 1)))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:472: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:473: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:474: FutureWarning: Passing (type, 1) or 

0 11.341459
200 13.776247
400 15.670144
600 15.341506
800 14.009693
1000 16.101307
1200 15.095751
1400 14.116156
1600 16.050594
1800 15.08965
2000 14.205284
[[9.9982268e-01 9.5705719e-09 1.7740109e-04]
 [9.8346680e-01 2.6015035e-04 1.6272981e-02]
 [7.8145581e-01 4.8756499e-02 1.6978768e-01]] [0 0 0]


In [3]:
import tensorflow as tf
import numpy as np
from google.colab import drive
drive.mount('/gdrive', force_remount=True)
# predicting animal type based on various features
xy = np.loadtxt('/gdrive/My Drive/Deep_learning_2102/data-04-zoo.csv', delimiter = ',', dtype = np.float32)
tf.set_random_seed(777)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]
nb_classes = 7 #0~6
X = tf.placeholder(tf.float32, [None, 16])
Y = tf.placeholder(tf.int32, [None, 1])

Y_one_hot = tf.one_hot(Y, nb_classes) #one_hot
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])

W = tf.Variable(tf.random_normal([16, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name = 'bias')

#tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
logits = tf.matmul(X, W) + b
hypothesis = tf.nn.softmax(logits)

#Cross entropy cost/loss
cost_i = tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = Y_one_hot)

cost = tf.reduce_mean(cost_i)
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.1).minimize(cost)

prediction = tf.argmax(hypothesis,1)
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#launch graph
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for step in range(2001) :
    sess.run(optimizer, feed_dict = {X: x_data, Y: y_data})
    loss, acc = sess.run([cost, accuracy], feed_dict = {X: x_data, Y: y_data})
    if step % 100 == 0:
      print("Step: {:5}\tLoss: {:.3f}\tAcc: {:.2%}".format(step, loss, acc))
  # Let's see if we can predict
  pred = sess.run(prediction, feed_dict = {X: x_data})
  # y_data: (N, 1) = flatten => (X, ) matches pred.shape
  for p, y in zip(pred, y_data.flatten()) :
    print("[{}] Prediction: {} True Y: {}".format(p == int(y), p, int(y)))

Mounted at /gdrive
Step:     0	Loss: 6.886	Acc: 12.87%
Step:   100	Loss: 0.601	Acc: 77.23%
Step:   200	Loss: 0.379	Acc: 84.16%
Step:   300	Loss: 0.272	Acc: 90.10%
Step:   400	Loss: 0.213	Acc: 93.07%
Step:   500	Loss: 0.176	Acc: 96.04%
Step:   600	Loss: 0.150	Acc: 99.01%
Step:   700	Loss: 0.131	Acc: 100.00%
Step:   800	Loss: 0.117	Acc: 100.00%
Step:   900	Loss: 0.105	Acc: 100.00%
Step:  1000	Loss: 0.096	Acc: 100.00%
Step:  1100	Loss: 0.088	Acc: 100.00%
Step:  1200	Loss: 0.081	Acc: 100.00%
Step:  1300	Loss: 0.075	Acc: 100.00%
Step:  1400	Loss: 0.070	Acc: 100.00%
Step:  1500	Loss: 0.066	Acc: 100.00%
Step:  1600	Loss: 0.062	Acc: 100.00%
Step:  1700	Loss: 0.059	Acc: 100.00%
Step:  1800	Loss: 0.056	Acc: 100.00%
Step:  1900	Loss: 0.053	Acc: 100.00%
Step:  2000	Loss: 0.051	Acc: 100.00%
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True